# Building a RAG System with Gemma, Elasticsearch and HuggingFace Models

In this example, we will build a RAG powered by Elasticsearch (ES) and HuggingFace models, letting us toggle between ES-vectorizing (our ES cluster vectorizes for us when ingesting and querying) vs self-vectorizing (we vectorize all our data before sending it to ES).

ES-vectorizing means our clients do not have to implement it, so that is the default here; however, if we do not have any ML nodes, or our own embedding setup is better/faster, we can set `USE_ELASTICSEARCH_VECTORIZATION = False` in the Section "Choose data and query vectorization options" below.

## Setups

In [ ]:
!pip install elasticsearch sentence_transformers transformers eland==8.12.1 # accelerate # uncomment if using GPU
!pip install datasets==2.19.2 # Remove version lock if https://github.com/huggingface/datasets/pull/6978 has been released

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

## Elasticsearch deployment

Make sure we have `CLOUD_ID` and `ELASTIC_DEPL_API_KEY` ready.

In [ ]:
from google.colab import userdata

# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#finding-your-cloud-id
CLOUD_ID = userdata.get("ELASTIC_CLOUD_ID")  # or "<YOUR CLOUD_ID>"

# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#creating-an-api-key
ELASTIC_API_KEY = userdata.get("ELASTIC_DEPL_API_KEY")  # or "<YOUR API KEY>"

Set up the client and make sure the credentials work.

In [ ]:
from elasticsearch import Elasticsearch, helpers

client = Elasticsearch(cloud_id=CLOUD_ID, api_key=ELASTIC_API_KEY)

client.info()

## Data sourcing and prepration

We will use the [`MongoDB/embedded_movies`](https://huggingface.co/datasets/MongoDB/embedded_movies) dataset sourced from HuggingFace datasets.

In [ ]:
from datasets import load_dataset

dataset = load_dataset('MongoDB/embedded_movies')

Next, we will do two things:
1. (Data integrity) we will ensure that each data point's `fullplot` attribute is not empty, as this is the primary data we utilize in the embedding process.
2. (Data quality) we will ensure that we remove the `plot_embedding` attribute from all data points as this will be replaced by new embeddings created with a different embedding model, the `gte-large`.

In [ ]:
# remove data point where plot column is missing
dataset = dataset.filter(lambda x: x['fullplot'] is not None)

# remove plot_embedding
if 'plot_embedding' in sum(dataset.column_names.values(), []):
    dataset = dataset.remove_columns('plot_embedding')

dataset['train']

In [ ]:
dataset['train'][0]

## Load Elasticsearch with vectorized data

### Choose data and query vectorization options

Here we need to make a decision: do we want Elasticsearch to vectorize our data and queries, or do we want to do it ourselves?

Setting `USE_ELASTICSEARCH_VECTORIZATION = True` will set up and use ES-hosted-vectorization for our data and our querying, but be aware that this requires our ES deployment to have at least 1 ML node.

If setting `USE_ELASTICSEARCH_VECTORIZATION = False`, then it will set up and use the provided model "locally" for data and query vectorization.

In this example, we picked the [`thenlper/gte-small`](https://huggingface.co/thenlper/gte-small) model for the embedding. Make sure that the `EMBEDDING_DIMENSIONS` is updated accordingly to the model.

In [ ]:
USE_ELASTICSEARCH_VECTORIZATION = True

EMBEDDING_MODEL_ID = 'thenlper/gte-small'
# https://huggingface.co/thenlper/gte-small's page shows the dimensions of the model
# If you use the `gte-base` or `gte-large` embedding models, the numDimension
# value in the vector search index must be set to 768 and 1024, respectively.
EMBEDDING_DIMENSIONS = 384

### Load HuggingFace model into Elasticsearch if needed

We can load and deploy the HuggingFace model into Elasticsearch using [Eland](https://eland.readthedocs.io/), if `USE_ELASTICSEARCH_VECTORIZATION = True`. This allows Elasticsearch to vectorize our queries and data in later steps.

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!(if [ "True" == $USE_ELASTICSEARCH_VECTORIZATION ]; then \
  eland_import_hub_model --cloud-id $CLOUD_ID --hub-model-id $EMBEDDING_MODEL_ID --task-type text_embedding --es-api-key $ELASTIC_API_KEY --start --clear-previous; \
fi)

This step adds functions for creating embeddings for text locally, and enriches the dataset with embeddings, so that the data can be ingested into Elasticsearch as vectors. Do not run if `USE_ELASTICSEARCH_VECTORIZATION = True`.

In [ ]:
from sentence_transformers improt SentenceTransformer

if not USE_ELASTICSEARCH_VECTORIZATION:
    embedding_model = SentenceTransformer(EMBEDDING_MODEL_ID)


def get_embedding(text: str) -> list[float]:
    if USE_ELASTICSEARCH_VECTORIZATION:
        raise Exception(f"Disable when USE_ELASTICSEARCH_VECTORIZATION = [{USE_ELASTICSEARCH_VECTORIZATION}]")
    else:
        if not text.strip():
            print('Attempted to get embedding for empty text.')
            return []

        embedding = embedding_model.encode(text)
        return embedding.tolist()

def add_fullplot_embedding(x):
    if USE_ELASTICSEARCH_VECTORIZATION:
        raise Exception(f"Disable when USE_ELASTICSEARCH_VECTORIZATION = [{USE_ELASTICSEARCH_VECTORIZATION}]")
    else:
        full_plots = x['fullplot']
        return {'embedding:' [get_embedding(full_plot) for full_plot in full_plot]}


if not USE_ELASTICSEARCH_VECTORIZATION:
    dataset = dataset.map(add_fullplot_embedding, batched=True)
    dataset['train']

## Create a search index with vector search mappings

Now, we can create an index in Elasticsearch with the right index mappings to handle vector searches.

In [ ]:
# Needs to match the id returned from Eland
# For HuggingFace models, we just replace the forward slash with double underscore
model_id = EMBEDDING_MODEL_ID.replace('/', '__')

index_name = 'movies'

index_mapping = {
    'properties': {
        'fullplot': {'type': 'text'},
        'plot': {'type': 'text'},
        'title': {'type': 'text'},
    }
}

# define index mapping
if USE_ELASTICSEARCH_VECTORIZATION:
    index_mapping['properties']['embedding'] = {
        'properties': {
            'is_truncated': {'type': 'boolean'},
            'model_id': {
                'type': 'text',
                'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}
            },
            'predicted_value': {
                'type': 'dense_vector',
                'dims': EMBEDDING_DIMENSIONS,
                'index': True,
                'similarity': 'cosine'
            }
        }
    }
else:
    index_mapping['properties']['embedding'] = {
        'type': 'dense_vector',
        'dims': EMBEDDING_DIMENSIONS,
        'index': True,
        'similarity': 'cosine'
    }

# flag to check if index has to be deleted before creating
should_delete_index = True

# check if we want to delete index before creating the index
if should_delete_index:
    if client.indices.exists(index=index_name):
        print(f'Deleting existing index {index_name}...')
        client.indices.delete(index=index_name, ignore=[400, 404])

print(f"Creating index {index_name}...")
# ingest pipeline definition
if USE_ELASTICSEARCH_VECTORIZATION:
    pipeline_id = 'vectorize_fullplots'

    client.ingest.put_pipeline(
        id=pipeline_id,
        processors=[
            {
                'inference': {
                    'model_id': model_id,
                    'target_field': 'embedding',
                    'field_map': {'fullplot': 'text_field'}
                }
            }
        ]
    )

    index_settings = {
        'index': {
            'default_pipeline': pipeline_id
        }
    }
else:
    index_settings = {}


client.options(ignore_status=[404, 400]).indices.create(
    index=index_name,
    mappings=index_mapping,
    settings=index_settings
)

Ingesting data into a Elasticsearch is best done in batches. We can use `helpers` to achieve this.

In [ ]:
from elasticsearch.helpers improt BulkIndexError


def batch_to_bulk_actions(batch):
    for record in batch:
        action = {
            '_index': 'movies',
            '_source': {
                'title': record['title'],
                'plot': record['plot'],
                'fullplot': record['fullplot']
            }
        }
        if not USE_ELASTICSEARCH_VECTORIZATION:
            action['_source']['embedding'] = record['embedding']
        yield action


def bulk_index(dataset):
    start = 0
    end = len(ds)
    batch_size = 100

    if USE_ELASTICSEARCH_VECTORIZATION:
        # If using auto-embedding, bulk requests can take a lot longer,
        # so we pass a longer request_timeout here (default to 10s),
        # otherwise we could get connection timeouts
        batch_client = client.options(request_timeout=600)
    else:
        batch_client = client

    for batch_start in range(start, end, batch_size):
        batch_end = min(batch_start + batch_size, end)
        print(f"batch: start [{batch_start}], end [{batch_end}]")
        batch = dataset.select(range(batch_start, batch_end))

        actions = batch_to_bulk_actions(batch)
        helpers.bulk(batch_client, actions)

In [ ]:
try:
    bulk_index(dataset['train'])
except BulkIndexError as e:
    print(f"{e.errors}")

print('Data ingestion into Elasticsearch complete.')

## Perform Vector Search on user queries

* If `USE_ELASTICSEARCH_VECTORIZATION = True`, the text query is sent directly to ES where the uploaded model will be used to vectorize it first before doing a vector search.
* If `USE_ELASTICSEARCH_VECTORIZATION = False`, we do the vectorization locally before sending a query with the vectorized form of the query.

In [ ]:
def vector_search(plot_query):
    if USE_ELASTICSEARCH_VECTORIZATION:
        knn = {
            'field': 'embedding.predicted_value',
            'k': 10,
            'query_vector_builder': {
                'text_embedding': {
                    'model_id': model_id,
                    'model_text': plot_query
                }
            },
            'num_candidates': 150,
        }
    else:
        question_embedding = get_embedding(plot_query)
        knn = {
            'field': 'embedding',
            'query_vector': question_embedding,
            'k': 10,
            'num_candidates': 150
        }

    response = client.search(index='movies', knn=knn, size=5)

    results = []
    for hit in response['hits']['hits']:
        id = hit['_id']
        score = hit['_score']
        title = hit['_source']['title']
        plot = hit['_source']['plot']
        full_plot = hit['_source']['fullplot']

        result = {
            'id': id,
            '_score': score,
            'title': title,
            'plot': plot,
            'full_plot': full_plot
        }
        results.append(result)

    return results

In [ ]:
def pretty_search(query):
    get_knowledge = vector_search(query)

    search_result = ""
    for result in get_knowledge:
        search_result += f"Title: {result.get('title', 'N/A')}, Plot: {result.get('fullplot', 'N/A')}\n"

    return search_result

## Handle user queries and load Gemma

In [ ]:
# Conduct query with retrival of sources,
# combining results into something we can feed to Gemma
def combined_query(query):
    source_information = pretty_search(query)
    return f"Query: {query}\nContinue to answer the query by using these Search Results:\n{source_information}"

In [ ]:
query = "What is the best romantic movie to watch and why?"
combined_results = combined_query(query)
print(combined_results)

Now we can load our LLM

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

llm_id = 'google/gemma-2b-it'

tokenizer = AutoTokenizer.from_pretrained(llm_id)
model = AutoModelForCausalLM.from_pretrained(llm_id, device_map='auto')

We need to define a method that fetches formatted results from a vectorized search in ES, and then feed it to the LLM to get our results.

In [ ]:
def rag_query(query):
    combined_information = combined_query(query)

    input_ids = tokenizer(
        combined_information,
        return_tensors='pt'
    ).to('cuda')
    response = model.generate(
        **input_ids,
        max_new_tokens=700
    )

    return tokenizer.decode(response[0], skip_special_tokens=True)

In [ ]:
print(rag_query(query))